# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opinii na stronie
    1. Skpiuj opinię
    2. Wklej opinię do edytora tekstu
3. Przejź do klejnej strony z opiami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami  

## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opini jej składowych 
    2. Dodanie do słownika reprezntującą pojedynczą opinię do listy

## Struktura opini w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id||
|autor|author||
|rekomendacja|recommendation||
|treść opinii|content||
|lista zalet|pros||
|lista wad|cons||
|data wystawienia opinii|post_date||
|data zakupu produktu|purchase_date||
|ile osób uznało opinię za przydatną|useful||
|ile osób uznało opinię za nieprzydatną|useless||

63717975
124893467
108481121
114228736
28020568

1. Import bibliotek

In [26]:
import requests
from bs4 import BeautifulSoup

2. Wysłanie żądania dostępu do zasobu (request)

In [34]:
product_code = "114228736"
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

3. Wydobycie z kodu HTML fragmentów odpowiadających opiniom

In [36]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")


4. Wydobycie z kodu opini jej składowych dla każdej opinii na stronie

In [37]:
import requests
from bs4 import BeautifulSoup
product_code = "114228736"
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
all_opinions=[]
for opinion in opinions:
    single_opinion = {
        'opinion_id': opinion["data-entry-id"].strip(),
        'author': opinion.select_one("span.user-post__author-name").text.strip(),
        'recommendation': opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
        'stars': opinion.select_one("span.user-post__score-count").text.strip(),
        'content': opinion.select_one("div.user-post__text").text.strip(),
        'pros': [p.text.strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        'cons': [c.text.strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        'post_date': opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
       # 'purchase_date': opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip(),
        'useful': opinion.select_one("button.vote-yes > span").text.strip(),
        'useless': opinion.select_one("button.vote-no > span").text.strip()
    }
    all_opinions.append(single_opinion)

AttributeError: 'NoneType' object has no attribute 'text'

5. Zapis opinii do JSON

In [39]:
import json, os 
if not os.path.exists("opinions"):
    os.mkdir("opinions")

jf = open(f"opinions/{product_code}.json","w",encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()